1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
# If haven't downloaded it yet, please get the data file with wget
!curl https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/
    
    
    
#invoc iwr
import pandas as pd
import numpy as np
import random as r

<a href="/s/raw/xvjzaxzz3ysphme/data_000637.txt">Found</a>.



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    61    0    61    0     0    130      0 --:--:-- --:--:-- --:--:--   131


1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [13]:
import pandas as pd
import random

data = pd.read_csv('data/data_000637.txt')
data = pd.DataFrame(data)
N = random.randint(10000, data.shape[0])

data = data[0:N]
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
638577,1,0,60,3869206570,3327,19
638578,1,1,17,3869206570,3321,20
638579,1,0,61,3869206570,3326,12
638580,1,1,4,3869206570,3322,9


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [24]:
bx = data['BX_COUNTER'] 
count = 0
for i in range(1,N):
    if bx[i]> bx[i-1]:
        count+=1
print(count)

401134


In [26]:
# prendo la colonna BX

BX = data.loc[:,['BX_COUNTER']]
print(BX)

x = max(BX.values)

print(x)


        BX_COUNTER
0             2374
1             2374
2             2553
3             2558
4             2760
...            ...
638577        3327
638578        3321
638579        3326
638580        3322
638581        3331

[638582 rows x 1 columns]
[3563]


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [47]:
import numpy as np
tdc = data['TDC_CHANNEL']*25/30
bx = data['BX_COUNTER'] *25
orbit = data['ORBIT_CNT']*25*count

data['Time_Series'] = tdc+bx+orbit


data['Time_Series'] -= data['Time_Series'].min()



In [61]:
data['TimeSeries'] = (data['ORBIT_CNT'] * x * 25) + (data['BX_COUNTER'] * 25) + data['TDC_MEAS'] * (25/30)

data['TimeSeries'] -= data['TimeSeries'].min()

data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time_Series,TimeSeries
0,1,0,123,3869200167,2374,26,1970-01-01 00:00:00.000000000,0.000000e+00
1,1,0,124,3869200167,2374,27,1970-01-01 00:00:00.000000000,8.125000e-01
2,1,0,63,3869200167,2553,28,1970-01-01 00:00:00.000004424,4.476625e+03
3,1,0,64,3869200167,2558,19,1970-01-01 00:00:00.000004544,4.594125e+03
4,1,0,64,3869200167,2760,25,1970-01-01 00:00:00.000009600,9.649125e+03
...,...,...,...,...,...,...,...,...
638577,1,0,60,3869206570,3327,19,1970-01-01 00:01:04.211548824,5.703710e+08
638578,1,1,17,3869206570,3321,20,1970-01-01 00:01:04.211548640,5.703709e+08
638579,1,0,61,3869206570,3326,12,1970-01-01 00:01:04.211548800,5.703710e+08
638580,1,1,4,3869206570,3322,9,1970-01-01 00:01:04.211548656,5.703709e+08


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [71]:
import datetime as dt
#data['Time_Series'] = pd.to_datetime(data['Time_Series'])
tstamp = pd.Timestamp(data.loc[N-1,'TimeSeries'])
print(tstamp)

duratao = int(durata)//3600
duratam = int((durata%3600)//60)
duratas = int(durata%60)
print(f"Data taking duration: {duratao} hours, {duratam} minutes, {duratas} seconds")


1970-01-01 00:00:00.570371134
Data taking duration: 0 hours, 1 minutes, 4 seconds


In [63]:
durata = data['TimeSeries'].max() - data['TimeSeries'].min() 
durata_ore = int(durata // 3600)
durata_minuti = int((durata % 3600) // 60)
durata_secondi = int(durata % 60)

print(f"Durata della raccolta dati: {durata_ore} ore, {durata_minuti} minuti, {durata_secondi} secondi")


Durata della raccolta dati: 158436 ore, 25 minuti, 34 secondi


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [82]:
df = (data.groupby('TDC_CHANNEL').size()).sort_values(ascending=False)
g = df.head(3)
print(g)

TDC_CHANNEL
139    53072
64     31984
63     31380
dtype: int64


In [78]:
# raggruppo in base al TCD_CHANNEL e conto quanti sono
noisy_channels = (data.groupby('TDC_CHANNEL').size()).sort_values(ascending=False)

top_noisy_channels = noisy_channels.head(3)

print("Top 3 Noisy Channels:")
print(top_noisy_channels)


Top 3 Noisy Channels:
TDC_CHANNEL
139    53072
64     31984
63     31380
dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [86]:
non_empty_orbits = data.groupby('ORBIT_CNT')['TDC_MEAS'].nunique() 

count_non_empty_orbits = non_empty_orbits[non_empty_orbits>0].count()

print("Number of Non-Empty Orbits:", count_non_empty_orbits)


Number of Non-Empty Orbits: 6400


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [100]:
df = data[data['TDC_CHANNEL'] == 139]
print(data)
unique_orbits_count = df['ORBIT_CNT'].nunique()


print("Number of Unique Orbits with at least one measurement from TDC_CHANNEL=139:", unique_orbits_count)

KeyError: 'Column not found: False'

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [123]:
data = pd.read_csv('data/data_000637.txt')
data = pd.DataFrame(data)
N = random.randint(10000, data.shape[0])

data = data[0:N]

df0 = data[data['FPGA'] == 0]
df1 =  data[data['FPGA'] == 1]

count0 = df0.groupby('TDC_CHANNEL').size()
print(count0)

f0= pd.Series(),index = )
#series_fpga1 = pd.Series(fpga1_data['count'].values, index=fpga1_data['TDC_channel'])

#TDC0 = data0.groupby('TDC_CHANNEL').count()
#TDC1 = data0.groupby('TDC_CHANNEL').size()
#TDC0
#df = pd.Series(, index = data0['TDC_CHANNEL'])

TDC_CHANNEL
1        736
2        951
3        915
4       1428
5        885
       ...  
129        2
130       20
137       19
138       20
139    46483
Length: 124, dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.

In [20]:


channel_counts = data.groupby(['FPGA', 'TDC_CHANNEL']).size().reset_index(name='count')

# Separate the counts for each FPGA
fpga1_counts = channel_counts[channel_counts['FPGA'] == 0].set_index('TDC_CHANNEL')['count']
fpga2_counts = channel_counts[channel_counts['FPGA'] == 1].set_index('TDC_CHANNEL')['count']

# Print or use the resulting Series as needed
print("FPGA1 Counts:")
print(fpga1_counts)

print("\nFPGA2 Counts:")
print(fpga2_counts)



FPGA1 Counts:
TDC_CHANNEL
1       139
2       170
3       173
4       240
5       177
       ... 
129       1
130       5
137       4
138       4
139    8722
Name: count, Length: 122, dtype: int64

FPGA2 Counts:
TDC_CHANNEL
1      3207
2      3686
3      2495
4      2990
5      1701
       ... 
129       4
130       6
137       4
138       4
139    3707
Name: count, Length: 132, dtype: int64
